**Testing a landing gear geometry**

Criteria :
- correct balance between main landing gear (MLG) and nose landing gear (NLG)
- limited overturn angle to ensure sufficient taxiing stability on sharp turns
- enough ground clearance for the aft mounted propeller

In [1]:
import numpy as np
from input import *